# Train model using Mouse Brain dataset

In this notebook, we're going to train our model using the Mouse Brain dataset (GSE60361). 

This assumes that you've made the graph using the ```Infer GRN.ipynb``` code.

In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Data, Dataset
from tqdm import tqdm
from datasets.datasetAtacSeqChromatin import AtacSeqChromatinDataset
from scipy.special import softmax
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tqdm import tqdm
from sklearn.metrics import (auc, precision_recall_curve, roc_auc_score,
                             roc_curve)
from statistics import mean

Torch version: 1.8.0+cu111
Cuda available: True
Torch geometric version: 2.0.3


Load up the dataset. Read ```datasetMouseBrain.py``` on how the dataset was built. 

In [2]:
dataset = AtacSeqChromatinDataset("/gpfs/data/rsingh47/hzaki1/atacseqdataChromatin")

100%|██████████| 103480/103480 [01:28<00:00, 1163.66it/s]


In [3]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: AtacSeqChromatinDataset(1047):
Number of graphs: 1047
Number of features: 2

Data(x=[18666, 2], edge_index=[2, 103480], y=[1])
Number of nodes: 18666
Number of edges: 103480
Average node degree: 5.54
Contains isolated nodes: True
Contains self-loops: False


/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


Is undirected: False


In [4]:
shuffle_index = np.loadtxt('shuffle_indices/atacseqShuffleIndex.txt')
shuffle_index = shuffle_index.astype(np.int32)
train_size, val_size = int(len(shuffle_index)* 0.8), int(len(shuffle_index)* 0.9)
train_dataset = [dataset[i] for i in shuffle_index[0:train_size]]
val_dataset = [dataset[i] for i in shuffle_index[train_size: val_size]]
test_dataset =  [dataset[i] for i in shuffle_index[val_size:]]

# train_dataset = torch.load('trainDataset.pt')
# test_dataset = torch.load('testDataset.pt')

# train_dataset = dataset[:837]
# test_dataset = dataset[837:]

# train_dataset = dataset[:40]
# test_dataset = dataset[40:60]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of validation graphs: {len(val_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 837
Number of validation graphs: 105
Number of test graphs: 105


In [5]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)
train_loader_testing = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

/gpfs/home/hzaki1/celltypefromgrn/env-gpu/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
from gcnmodel import GCN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCN(hidden_channels=128, data=dataset, output_size=4).to(device)
print(model)

GCN(
  (conv1): SAGEConv(2, 128)
  (conv2): SAGEConv(128, 128)
  (conv3): SAGEConv(128, 128)
  (lin): Linear(in_features=128, out_features=4, bias=True)
)


In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.CrossEntropyLoss()
    
def test(loader, size):
    model.eval()
    output = np.zeros((len(loader), size))
    actual = np.zeros((len(loader), size))
    accuracy = 0
    for ind, data in enumerate(loader):  # Iterate in batches over the training/test dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 2))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        data.x[[0,1],1] = data.x[[1,0],1]
        out = model(data.x, data.edge_index, data.batch)
        output[ind] = softmax(out.cpu().detach().numpy())
        actual[ind][data.y] = 1
        accuracy += int((out.argmax(dim=1) == data.y).sum())
    all_labels = list(dataset.cellToIndex.keys())
    actual = np.array(actual)
    precision = dict()
    recall = dict()
    averageAUROC = []
    averageAUPR = []
    for (idx, c_label) in enumerate(all_labels):
        
        fpr, tpr, thresholds = roc_curve(actual[:,idx].astype(int), output[:,idx])
        precision[idx], recall[idx], _ = precision_recall_curve(actual[:, idx],
                                                        output[:, idx])
        averageAUROC.append(auc(fpr, tpr))
        averageAUPR.append(round(auc(recall[idx], precision[idx]),4))

    return accuracy/len(loader.dataset), mean(averageAUROC), mean(averageAUPR)


def train():
    model.train()
    avgLoss = 0
    for data in tqdm(train_loader, total=28):  # Iterate in batches over the training dataset.
        data.x = torch.reshape(data.x, (data.x.shape[0], 2))
        data.x = data.x.type(torch.FloatTensor)
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)# Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        avgLoss += loss
    return avgLoss / 28


for epoch in range(1, 150):
        loss = train()
        train_acc, trainAUC, trainAUPR = test(train_loader_testing, 4)
        test_acc,testAUC, testAUPR = test(test_loader, 4)
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train AUC: {trainAUC:.4f}, Train AUPR: {trainAUPR:.4f}, Test Acc: {test_acc:.4f}, Test Auc: {testAUC:.4f}, Test AUPR: {testAUPR:.4f},  Loss: {loss:.4f}')

100%|██████████| 28/28 [00:17<00:00,  1.64it/s]


Epoch: 001, Train Acc: 0.3859, Train AUC: 0.6663, Train AUPR: 0.3924, Test Acc: 0.4095, Test Auc: 0.6786, Test AUPR: 0.3974,  Loss: 1.3101


100%|██████████| 28/28 [00:08<00:00,  3.29it/s]


Epoch: 002, Train Acc: 0.4313, Train AUC: 0.6830, Train AUPR: 0.4098, Test Acc: 0.4190, Test Auc: 0.6753, Test AUPR: 0.3802,  Loss: 1.2746


100%|██████████| 28/28 [00:07<00:00,  3.84it/s]


...

Epoch: 147, Train Acc: 0.9355, Train AUC: 0.9951, Train AUPR: 0.9839, Test Acc: 0.6381, Test Auc: 0.8505, Test AUPR: 0.6416,  Loss: 0.3723


100%|██████████| 28/28 [00:06<00:00,  4.14it/s]


Epoch: 148, Train Acc: 0.9367, Train AUC: 0.9945, Train AUPR: 0.9820, Test Acc: 0.6476, Test Auc: 0.8502, Test AUPR: 0.6256,  Loss: 0.3177


100%|██████████| 28/28 [00:05<00:00,  4.75it/s]


Epoch: 149, Train Acc: 0.9200, Train AUC: 0.9934, Train AUPR: 0.9785, Test Acc: 0.6476, Test Auc: 0.8413, Test AUPR: 0.6294,  Loss: 0.3233


In [8]:
torch.save(model.state_dict(), 'model_weightsjun3_atacseqChromatin_data.pth')